<a href="https://colab.research.google.com/github/datascience-uniandes/data-analysis-tutorial/blob/master/airbnb/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 1

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
**Dataset:** Hotel bookings [[dataset](https://drive.google.com/file/d/1VA3XyONx5mFEzx1YN_ZsPAkEMWXuRBqI/view?usp=sharing) | [dictionary](https://docs.google.com/spreadsheets/d/1p-lHMlXSUAYdP19odh3AqqkowHRWfWSHWXCURqMOmCc/edit?usp=sharing)].


Última actuaización: Septiembre, 2025

In [ ]:
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Pandas configuration for extending the number of columns and rows to show
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

### 1. Entendimiento inicial de datos

In [ ]:
hotel_file_url = 'https://raw.githubusercontent.com/lojedaa/Talleres-Ciencia-de-Datos/refs/heads/main/Taller_1/hotel_bookings_modified.csv'
hotel_dict_url = 'https://raw.githubusercontent.com/lojedaa/Talleres-Ciencia-de-Datos/refs/heads/main/Taller_1/hotel_dictionary.csv'

hotel_df = pd.read_csv(hotel_file_url, low_memory=False)
hotel_dict = pd.read_csv(hotel_dict_url, sep=',', header=0)

In [ ]:
# Crear diccionario de tipos
varType = dict(zip(hotel_dict['Field'], hotel_dict['Type']))

# Función para convertir tipos
def Type_convert(var_type):
    if var_type == 'integer':
        return 'Int64'
    elif var_type == 'numeric':
        return 'float'
    elif var_type == 'text':
        return 'string'
    elif var_type == 'boolean':
        return 'boolean'
    elif var_type == 'date':
        return 'datetime64[ns]'
    else:
        return None  # tipo desconocido

In [ ]:
# Aplicar conversión de tipos
for col, tipo in varType.items():
    tipo_convertido = Type_convert(tipo)
    if tipo_convertido == 'datetime64[ns]':
        hotel_df[col] = pd.to_datetime(hotel_df[col], errors='coerce')
    elif tipo_convertido == 'Int64':
        hotel_df[col] = pd.to_numeric(hotel_df[col], errors='coerce').astype('Int64')
    elif tipo_convertido:
        hotel_df[col] = hotel_df[col].astype(tipo_convertido)



In [ ]:
# Showing dataframe dimensions
hotel_df.shape

In [ ]:
# Showing column types
hotel_df.dtypes

,0
hotel,string[python]
is_canceled,boolean
lead_time,Int64
arrival_date_year,Int64
arrival_date_month,string[python]
arrival_date_week_number,Int64
arrival_date_day_of_month,Int64
stays_in_weekend_nights,Int64
stays_in_week_nights,Int64
adults,Int64


In [ ]:
hotel_df.sample(10) # Showing a sample of 10 rows

### 1.1. Análisis univariado

In [ ]:
def univariate_summary(df, column):
    """Generate comprehensive univariate summary for a column"""
    print(f"\n{'='*50}")
    print(f"UNIVARIATE ANALYSIS: {column}")
    print(f"{'='*50}")

    # Basic info
    print(f"Data type: {df[column].dtype}")
    print(f"Total values: {len(df[column])}")
    print(f"Missing values: {df[column].isnull().sum()} ({df[column].isnull().mean()*100:.2f}%)")
    print(f"Unique values: {df[column].nunique()}")

    if df[column].dtype in ['Int64', 'float']:
        # Numerical analysis
        print(f"\n--- Numerical Statistics ---")
        print(df[column].describe())
        print(f"Skewness: {df[column].skew():.3f}")
        print(f"Kurtosis: {df[column].kurtosis():.3f}")

    elif df[column].dtype == 'object' or df[column].nunique() < 15:
        # Categorical analysis
        print(f"\n--- Frequency Distribution ---")
        print(df[column].value_counts())
        print(f"\n--- Percentage Distribution ---")
        print(df[column].value_counts(normalize=True) * 100)

# Example usage
univariate_summary(hotel_df, 'hotel')
univariate_summary(hotel_df, 'customer_type')
univariate_summary(hotel_df, 'distribution_channel')
univariate_summary(hotel_df, 'assigned_room_type')
univariate_summary(hotel_df, 'arrival_date_month')
univariate_summary(hotel_df, 'is_canceled')


In [ ]:
# Plotting bar charts for arrival_date_month and is_canceled and distribution_channel
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, sharey=True, figsize=(20, 5))
hotel_df["arrival_date_month"].value_counts().plot(kind="bar", ax=ax1, color="orange")
hotel_df["is_canceled"].value_counts().sort_index(ascending=False).plot(kind="bar", ax=ax2, color="green")
hotel_df["distribution_channel"].value_counts().plot(kind="bar", ax=ax3, color="steelblue")
ax1.set_title("arrival_date_month")
ax2.set_title("is_canceled")
ax3.set_title("distribution_channel")
plt.tight_layout()
plt.show()

In [ ]:
# Armando un boxplot para distribución por número de adultos
plt.figure(figsize=(20, 5))
plt.boxplot(hotel_df["adults"], showmeans=True, vert=False)
plt.title("Distribución por número de adultos")
plt.show()

### 2. Estrategia de Análisis

In [ ]:
# Plotting correlation heatmap sobre grupo huéspedes
sns.heatmap(
    hotel_df[["adults", "children", "babies", "is_canceled","adr"]].corr(),
    vmin=0.5, vmax=1,
    cmap="Blues"
)
plt.title("Correlación sobre grupo de huéspedes")
plt.show()